In [ ]:
import os
import glob
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
# import shutil # ダミーデータ生成時のみ使用

# ==========================================
# 1. ダミーデータ生成パート (学習部分想定)
# ※ ユーザーの環境では既にモデルがあるため、この部分は不要ですが
#    動作確認用にコードのみ残してコメントアウト状態で置いておきます。
# ==========================================
# class MockStatsModel:
#     """statsmodelsの結果を模倣したダミークラス"""
#     def __init__(self, variables):
#         # 係数 (beta/weight)
#         self.params = pd.Series(np.random.randn(len(variables)), index=variables)
#         # 分散共分散行列の対角成分（分散）を模倣 (標準誤差 bse の2乗想定)
#         self.bse = pd.Series(np.abs(np.random.randn(len(variables)) * 0.1), index=variables)
#
# def generate_mock_data(folder_path="models_folder"):
#     """テスト用のpickleファイルを生成する（学習プロセスの代わり）"""
#     if os.path.exists(folder_path):
#         shutil.rmtree(folder_path)
#     os.makedirs(folder_path)
#
#     base_vars = ["const", "GDP", "CPI"]
#     extra_vars = ["Unemployment", "InterestRate", "ExchangeRate"]
#
#     # 2020年1月から月次でスライディングウィンドウしたと仮定
#     start_date = datetime(2020, 1, 1)
#
#     print(f"--- Generating mock data in '{folder_path}' ---")
#     for i in range(12):
#         # 期間が進むにつれて変数が変わるシチュエーション
#         current_vars = base_vars.copy()
#         if i > 2: current_vars.append(extra_vars[0])
#         if i > 5: current_vars.append(extra_vars[1])
#         if i > 8: current_vars.remove("GDP") # 変数が減るパターン
#
#         model = MockStatsModel(current_vars)
#
#         # ファイル名形式: exp1_ver1_YYYYMMDD.pkl
#         date_str = (start_date + pd.DateOffset(months=i)).strftime('%Y%m%d')
#         filename = f"exp1_ver1_{date_str}.pkl"
#
#         with open(os.path.join(folder_path, filename), 'wb') as f:
#             pickle.dump(model, f)
#
#     print("Generation complete.\n")


# ==========================================
# 2. メイン処理: モデル読み込みとデータフレーム化
# ==========================================

def extract_date_from_filename(filename):
    """
    ファイル名から日付らしき数字を抽出してソート用に返す関数
    例: 'exp1_ver1_201101.pkl' -> '201101'
    """
    # ファイル名から6桁～8桁の連続する数字を探す (YYMMDD, YYYYMMDDなど)
    match = re.search(r'(\d{6,8})', filename)
    if match:
        return match.group(1)
    else:
        # 数字が見つからない場合はファイル名そのまま
        return filename

def load_and_process_models(folder_path):
    """
    指定フォルダ内の全pklファイルを日付順に読み込み、
    係数と分散を抽出してDataFrameにする
    """
    # pklファイルを取得
    files = glob.glob(os.path.join(folder_path, "*.pkl"))

    if not files:
        print(f"Error: No .pkl files found in {folder_path}")
        return pd.DataFrame()

    # ファイル名に含まれる日付数字でソートする
    # これにより exp1_ver1_201101, exp1_ver1_201102... の順序を保証
    files.sort(key=lambda x: extract_date_from_filename(os.path.basename(x)))

    all_records = []
    print(f"Found {len(files)} models. Starting extraction...")

    for file_path in files:
        filename = os.path.basename(file_path)

        # 識別子として日付部分を取得、なければファイル名全体
        period_id = extract_date_from_filename(filename)

        try:
            with open(file_path, 'rb') as f:
                model = pickle.load(f)

            # --- データの抽出ロジック (statsmodels / 汎用) ---

            # 1. 変数名と係数の取得
            variables = []
            coeffs = []
            variances = []

            # パターンA: statsmodels (paramsがSeriesで変数名を持っている)
            if hasattr(model, 'params') and isinstance(model.params, pd.Series):
                variables = model.params.index.tolist()
                coeffs = model.params.values

                # 分散の取得 (bse^2 または cov_paramsの対角成分)
                if hasattr(model, 'bse'):
                    # bseは標準誤差なので2乗して分散にする
                    variances = model.bse.values ** 2
                elif hasattr(model, 'cov_params'):
                    variances = np.diag(model.cov_params())
                else:
                    variances = [np.nan] * len(variables)

            # パターンB: sklearn等 (coef_がarray, feature_namesがある場合など)
            elif hasattr(model, 'coef_'):
                coeffs = model.coef_
                # 変数名 (持っていない場合は連番)
                if hasattr(model, 'feature_names_in_'):
                    variables = model.feature_names_in_
                else:
                    variables = [f'var_{i}' for i in range(len(coeffs))]

                variances = [np.nan] * len(variables) # sklearnは分散を直接持たないことが多い

            # パターンC: 辞書型などで保存されている場合 (カスタム)
            elif isinstance(model, dict):
                # 'weights', 'beta', 'params' などのキーを探す簡易ロジック
                for key in ['params', 'weights', 'beta', 'coef']:
                    if key in model:
                        # ここは辞書の構造次第なので簡易的な対応です
                        if isinstance(model[key], dict): # {var: val}形式
                            variables = list(model[key].keys())
                            coeffs = list(model[key].values())
                        else: # リスト形式
                            coeffs = model[key]
                            variables = [f'var_{i}' for i in range(len(coeffs))]
                        break
                variances = [np.nan] * len(variables)

            else:
                print(f"Warning: Could not extract data from {filename} (Unknown format)")
                continue

            # リストに追加 (Long Format)
            for var, coef, var_val in zip(variables, coeffs, variances):
                all_records.append({
                    'Period': period_id,   # 日付/識別子
                    'Variable': var,       # 変数名
                    'Coefficient': coef,   # 推定量 β
                    'Variance': var_val,   # 分散
                    'SourceFile': filename
                })

        except Exception as e:
            print(f"Error reading {filename}: {e}")

    # データフレーム作成
    df = pd.DataFrame(all_records)
    return df

# ==========================================
# 3. 可視化とCSV出力
# ==========================================

def export_and_visualize(df, output_folder="output_results"):
    """
    データフレームを受け取り、CSV保存とグラフ描画を行う
    """
    if df.empty:
        print("Dataframe is empty. Nothing to export.")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # ---------------------------------------
    # 1. CSV出力
    # ---------------------------------------

    # (A) Long Format (データベース保存用などに適した形式: 期間, 変数, 値)
    long_csv_path = os.path.join(output_folder, "model_params_long.csv")
    df.to_csv(long_csv_path, index=False)
    print(f"Saved Long format CSV: {long_csv_path}")

    # (B) Wide Format (ピボット形式: 横軸に期間、縦軸に変数、値は係数)
    # Excelで見やすい形式
    try:
        # 係数 (Coefficient) の一覧
        df_coef = df.pivot(index='Period', columns='Variable', values='Coefficient')
        coef_csv_path = os.path.join(output_folder, "model_coefficients_pivot.csv")
        df_coef.to_csv(coef_csv_path)
        print(f"Saved Coefficients Pivot CSV: {coef_csv_path}")

        # 分散 (Variance) の一覧
        df_var = df.pivot(index='Period', columns='Variable', values='Variance')
        var_csv_path = os.path.join(output_folder, "model_variances_pivot.csv")
        df_var.to_csv(var_csv_path)
        print(f"Saved Variances Pivot CSV: {var_csv_path}")

    except ValueError as e:
        print(f"Pivot Error: {e} (Possibly duplicate entries for same period/variable)")

    # ---------------------------------------
    # 2. 可視化 (時系列プロット)
    # ---------------------------------------
    # 日本語フォント設定 (必要に応じてコメントアウトを外してパスを指定してください)
    # plt.rcParams['font.family'] = 'MS Gothic'

    sns.set_theme(style="whitegrid")

    # (A) 係数の推移プロット
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=df, x='Period', y='Coefficient', hue='Variable', marker='o')
    plt.title('Time Series of Coefficients (Beta)')
    plt.xlabel('Period (Date)')
    plt.ylabel('Coefficient Value')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, "plot_coefficients.png"))
    plt.close() # メモリ解放

    # (B) 分散の推移プロット
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=df, x='Period', y='Variance', hue='Variable', linestyle='--', marker='x')
    plt.title('Time Series of Estimation Variance')
    plt.xlabel('Period (Date)')
    plt.ylabel('Variance')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, "plot_variances.png"))
    plt.close()

    print(f"Plots saved in '{output_folder}'")


# ==========================================
# メイン実行ブロック
# ==========================================
if __name__ == "__main__":
    # ---------------------------------------------------------
    # 設定: ここを実際の環境に合わせて変更してください
    # ---------------------------------------------------------

    # モデルの入っているフォルダパス
    MODELS_DIR = "models_folder"

    # 出力先フォルダ
    OUTPUT_DIR = "analysis_results"

    # ---------------------------------------------------------
    # 実行処理
    # ---------------------------------------------------------

    # 学習部分（ダミーデータ生成）はコメントアウト済み
    # generate_mock_data(MODELS_DIR)

    print(f"Processing models in: {MODELS_DIR}")

    # 1. 読み込みと整形
    df_results = load_and_process_models(MODELS_DIR)

    if not df_results.empty:
        # 2. CSV出力と可視化
        export_and_visualize(df_results, OUTPUT_DIR)

        print("\nDone. Check the output folder.")
        print(df_results.head())
    else:
        print("No data was processed.")